### HEALTH EQUITY INDICATORS; SAN FRANCISCO, CA 2020 ###

In this notebook, I will combine the data I have collected and analyzed thus far on COVID-19 burden and access to health insurance in San Francisco. I am going to add information on hospital locations in the city and look at the proximity of hospitals to the neighborhoods that are most affected by COVID-19. 

#### IMPORTING DATA ####

In [2]:
#importing libraries
import pandas as pd
import geopandas as gpd
import plotly.express as px

First, I am going to pull in data that will connect the census tracts with actual recognized neighborhoods in San Francisco.

In [3]:
#pull in the Analysis Neighborhoods data set from DataSF
nb = gpd.read_file('W)

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


DriverError: W7 Assignment 3/data/SF2020_Neighborhoods_CTs.geojson: No such file or directory

In [ ]:
nb.shape

It looks like there are 242 census tracts which aligns with the data sets I have been working with.

In [ ]:
nb.head(5)

I will now clean up this data a bit and drop some of the census tracts that correspond to islands with no residents.

In [ ]:
#dropping the Farrollones census tract because it has messed with our maps in the past
nb = nb.drop(labels=3, axis=0)

In [ ]:
nb.shape

Ok, it appears the tract was dropped!

In [ ]:
#renaming the six columns
nb.columns = ['Name',
              'Neighborhood', 
              'Census Tract',
              'Supervisor District 2022',
              'Data Loaded',
              'County Code',
              'State Code',
              'Supervisor District 2012',
              'Data as of',
              'Object ID',
              'FIPS',
              'Geometry',]

In [ ]:
#checking that the above function worked
nb.head()

In [ ]:
# getting a count on how many census tracts are in each neighborhood
nb['Neighborhood'].value_counts()

Great, I am excited to see how this overlays with the health indicator data!

But before that happend, I need to join this data with the spatial data from 2020 census tracts. Then, I will start to map.

In [ ]:
# bring in the geojson file with census tracts
tracts=gpd.read_file('W7 Assignment 3/data/2020CTs.geojson')
# see what the data looks like
tracts.head()

In [ ]:
#seeing how many rows and columns we're working with 
tracts.shape

This is almost a match with the 241 tracts from the neighborhood data set. But I will pull a bit more info before merging this with the neighborhood data.

In [ ]:
tracts.info(5)

Based on this list of columns, we really only need the census tract number (tractce10) and geometry columns in order to merge the two data sets. Let's move forward with doing so.

In [ ]:
tracts = tracts[['tractce','geometry']]
tracts.head()

We have the census tract number, but we need a full FIPS code to merge. We'll add '06' for the state of California and '075' for the county of San Francisco to the census tract number as follows.

In [ ]:
# create a FIPS column before merging
tracts['FIPS'] ='06' + '075' + tracts['tractce']

In [ ]:
# double check that we have a new and complete FIPS code column
tracts.head()

Great, that worked! Now the FIPS codes will match the structure of the FIPS codes from the other data set which will make it possible to merge the two files.

In [ ]:
# create a new dataframe based on the join, tracts is spatial data so must come first
nhoods=tracts.merge(nb,on="FIPS")

In [ ]:
nhoods.head(5)

In [ ]:
# define the columns to keep, which include: geometry, FIPS and Neighborhood
columns_to_keep = ['geometry',
                   'FIPS',
                   'Neighborhood']

In [ ]:
# copy and create a new function, nhoods2, that just displays the defined columns to keep
nhoods2 = nhoods[columns_to_keep].copy()
nhoods2

This is exactly what we need in order to start mapping our neighborhoods of census tracts! In the next section, I will import the COVID-19 data and then the health insurance data and then map it over the neighborhoods. But first, I want to see what the neighborhoods look like mapped out!

In [ ]:
#simple plot map with colors
nhoods.plot(column='Neighborhood')

### COVID-19 DATA ###

In [ ]:
# import the data and create it as a new variable
COVIDdf = pd.read_csv('W7 Assignment 3/data/COVID-19 Data SF.csv')

In [ ]:
COVIDdf.info

Great news, there are 241 rows AKA 241 census tracts. This matches the neighborhood data!

In [ ]:
# check for leading zeros
COVIDdf['id'].sample(5)

In [ ]:
# adding leading zeros because I can see this column does not have them
COVIDdf = pd.read_csv(
    'W7 Assignment 3/data/COVID-19 Data SF.csv',
    dtype=
    {
        'id':str,
    }
)

In [ ]:
# making sure leading zeros made it in
COVIDdf['id'].sample(5)

In [ ]:
COVIDdf.head(5)

Now that the COVID-19 data is uploaded, I will try to merge the COVID-19 data with the neighborhood data. 

In [ ]:
#renaming the COVID-19 data file column from id to FIPS
COVIDdf = COVIDdf.rename(columns={'id': 'FIPS'})

In [ ]:
COVIDdf.head(5)

In [ ]:
# create a new dataframe based on the join
tracts_COVID=nhoods2.merge(COVIDdf,on="FIPS")

In [ ]:
# what does it look like now?
tracts_COVID.head()

Now I will make a bar chart that shows the 5 census tracts in SF with the highest rate of COVID-19 cases and highest number of COVID-19 deaths.

In [ ]:
COVIDdf=COVIDdf.sort_values(by='rate',ascending = False)

In [ ]:
# creating a bar chart
COVIDdf.head(5).plot.barh(x='FIPS',
    y='rate', 
    title='Top 5 Census Tracts in SF with Highest Rate of COVID-19 Cases')

In [ ]:
COVIDdf=COVIDdf.sort_values(by='deaths',ascending = False)

In [ ]:
# creating a bar chart
COVIDdf.head(5).plot.barh(x='FIPS',
    y='deaths', 
    title='Top 5 Census Tracts in SF with Highest Number of COVID-19 Deaths')

### MAPPING ###

In [ ]:
import folium

In [ ]:
# mpaping COVID case count by neighborhood
m = folium.Map(location=[37.7,-122.4],
zoom_start = 12,
tiles='CartoDB positron',
attribution='CartoDB')

# plot chorpleth over the base map
folium.Choropleth(
    geo_data=tracts_COVID, # geo data
    data=tracts_COVID, # data
    key_on='feature.properties.FIPS', # key, or merge column
    columns=['FIPS', 'rate'], # [key, value]
    fill_color='BuPu',
    line_weight=0.1,
    fill_opacity=0.8,
    line_opacity=0.2, # line opacity (of the border)
    legend_name='COVID Rate by Census Tract, SF').add_to(m)

# name on the legend color bar
m

In [ ]:
# mapping COVID rate by neighborhood
m = folium.Map(location=[37.7,-122.4],
zoom_start = 12,
tiles='CartoDB positron',
attribution='CartoDB')

# plot chorpleth over the base map
folium.Choropleth(
    geo_data=tracts_COVID, # geo data
    data=tracts_COVID, # data
    key_on='feature.properties.FIPS', # key, or merge column
    columns=['FIPS', 'count'], # [key, value]
    fill_color='BuPu',
    line_weight=0.1,
    fill_opacity=0.8,
    line_opacity=0.2, # line opacity (of the border)
    legend_name='COVID Count by Census Tract, SF').add_to(m)

# name on the legend color bar
m

### HEALTH INSURANCE DATA ###

In [ ]:
# importing health insurance coverage data
HIdf = pd.read_csv('W7 Assignment 3/data/Health Insurance.csv')

In [ ]:
HIdf.shape